In [1]:
import wandb

In [2]:
# pip install wandb

In [3]:
# pip install setuptools==58

In [4]:
# pip install pygooglenews --upgrade

In [5]:
# pip install --upgrade feedparser

In [6]:
from pygooglenews import GoogleNews
gn = GoogleNews()

In [7]:
headline_list= []
crypto_list = []

CRYPTOs = ['bitcoin' ,'eth', 'cardano', 'safemoon', 'altcoin', 'litecoin', 'fantom' ]
for crypto in CRYPTOs:
    response = gn. search (crypto)
    for r in response['entries']:
        headline_list.append(r['title'])
        crypto_list.append (crypto)

In [8]:
len(headline_list)

694

In [9]:
# pip install transformers

In [10]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained ( "ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

/Users/firex/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/firex/anaconda3/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [11]:
import wandb
wandb.init(project="CryptoCurrency Headline_Sentiment_Analysis")
headlines_table = wandb.Table(columns=["Headline", "Crypto", "Positive", "Negative", "Neutral"] )

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: anthonybreeganzo02 (ai_hackathon). Use `wandb login --relogin` to force relogin


In [12]:
import torch
def chunkList(list,n) :
    for i in range(0, len(list),n):
        yield list[i:i+n]
n=0
chunkSize = 100
model.eval()
for lines, cryptos in zip(chunkList (headline_list,chunkSize), chunkList (crypto_list, chunkSize) ):
    input = tokenizer (lines, padding=True, truncation=True, return_tensors='pt')
    outputs = model (**input)
    prediction = torch.nn.functional.softmax(outputs.logits,dim=-1)
    print(f" {n+1}/{int(len (headline_list)/chunkSize)}")
    for headline, crypto, pos, neg, neutr in zip(lines, cryptos, prediction[:,0].tolist(), prediction[:, 1]. tolist (), prediction[:, 1]. tolist ()):
        headlines_table.add_data (headline,crypto,pos,neg, neutr)
        n=n+1
print(type(headlines_table))

 1/6
 101/6
 201/6
 301/6
 401/6
 501/6
 601/6
<class 'wandb.data_types.Table'>


In [13]:
wandb.run.log({"Cryptocurrency Sentiment Analysis Table ": headlines_table})
wandb.run.finish()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


In [14]:
import pandas as pd
table_data = headlines_table.data
df = pd.DataFrame(table_data, columns=headlines_table.columns)
df.to_csv("/Users/firex/Hackathon/aihackathon24/AI_HFT_Quantum_Project/data/processed/Setimental_Analysis.csv", index=False)

In [15]:
df.head(10)

,Headline,Crypto,Positive,Negative,Neutral
0,"Bitcoin Halving: What It Means, Potential Impa...",bitcoin,0.032197,0.264306,0.264306
1,Bitcoin Miners Are Devouring Energy at a Recor...,bitcoin,0.056784,0.043786,0.043786
2,How to capitalize on the Bitcoin halving - Coi...,bitcoin,0.118363,0.011485,0.011485
3,"Bitcoin breaks $70,000 in volatile trading, hi...",bitcoin,0.512938,0.071788,0.071788
4,Bitcoin’s black swan event: How the return of ...,bitcoin,0.055200,0.043232,0.043232
5,SEC's Gensler says crypto 'rife with abuses an...,bitcoin,0.440768,0.169024,0.169024
6,‘More Dangerous’ Than Nukes—China And Russia R...,bitcoin,0.094116,0.222146,0.222146
7,Why Bitcoin Just Hit Its All - TIME,bitcoin,0.245069,0.034447,0.034447
8,"This Week in Coins: Bitcoin Touches $70,000 As...",bitcoin,0.445889,0.074235,0.074235
9,"Bitcoin bulls eye $100,000 as the next level b...",bitcoin,0.077182,0.018737,0.018737
